# Read Blast files

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns   

In [9]:
# Files to process
blast_files = [
    "gill_100seqs_randinit.txt",
    "gill_100seqs_initseq.txt",
    "mc_100seqs_randinit.txt",
    "mc_100seqs_initseq.txt",
    "plm_100seqs_randinit.txt",
    "plm_100seqs_initseq.txt"
]

summary_records = []

for filename in blast_files:
    df = pd.read_csv(filename, sep='\t', comment='#', header=None)
    df.columns = [
        "query", "subject", "identity", "alignment_length", "mismatches", "gap_opens",
        "q_start", "q_end", "s_start", "s_end", "evalue", "bit_score", "positives"
    ]

    best_hits = df.sort_values(by=["query", "identity"], ascending=[True, False]).groupby("query").first()

    identity_array = best_hits['identity'].to_numpy()
    alignment_length_array = best_hits['alignment_length'].to_numpy()
    bit_score_array = best_hits['bit_score'].to_numpy()

    # Handle zero e-values by replacing them with a very small number to avoid -inf after log10
    evalue_array = best_hits['evalue'].replace(0, 1e-300).to_numpy()
    log_evalues = -np.log10(evalue_array)

    # Calculate statistics
    identity_mean = np.mean(identity_array)
    identity_std = np.std(identity_array)

    alignment_length_mean = np.mean(alignment_length_array)
    alignment_length_std = np.std(alignment_length_array)

    bit_score_mean = np.mean(bit_score_array)
    bit_score_std = np.std(bit_score_array)

    log_evalue_mean = np.mean(log_evalues)
    log_evalue_std = np.std(log_evalues)

    # Create friendly model/init name
    readable_name = filename.replace("100seqs_", "").replace(".txt", "")

    summary_records.append({
        "Model": readable_name,
        "Mean Identity": identity_mean,
        "Std Identity": identity_std,
        "Mean Alignment Length": alignment_length_mean,
        "Std Alignment Length": alignment_length_std,
        "Mean Bit Score": bit_score_mean,
        "Std Bit Score": bit_score_std,
        "Mean -log10(E-value)": log_evalue_mean,
        "Std -log10(E-value)": log_evalue_std
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_records)

# Save CSV file
summary_file = "blast_summary_table.csv"
summary_df.to_csv(summary_file, index=False)

# Display nicely rounded table in Jupyter Notebook
display(summary_df.round(2))

,Model,Mean Identity,Std Identity,Mean Alignment Length,Std Alignment Length,Mean Bit Score,Std Bit Score,Mean -log10(E-value),Std -log10(E-value)
0,gill_randinit,58.04,5.60,60.97,3.25,71.31,8.68,11.81,3.16
1,gill_initseq,57.28,6.56,54.57,8.26,62.86,13.22,8.80,4.69
2,mc_randinit,61.75,7.15,61.79,3.01,76.96,12.96,13.89,4.69
3,mc_initseq,63.80,6.95,61.29,2.92,80.44,11.85,15.05,4.30
4,plm_randinit,64.85,6.80,61.88,2.00,81.94,11.91,15.73,4.25
5,plm_initseq,65.28,6.31,61.65,2.65,82.01,10.15,15.78,3.74


In [11]:
with open("blast_summary_table.tex", "w") as f:
    f.write(summary_df.round(2).to_latex(index=False))